In [229]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from wordcloud import WordCloud
from PIL import Image
import json

In [397]:
df = pd.read_csv("../data/billboard.csv", index_col = 0)
df = df[df.date >= "2000-01-01"]
df = df[df.date < "2022-01-01"]

In [398]:
def date_to_ym(x):
    y, m, d = x.split('-')
    return float(y+'.' + str(m))

In [399]:
# df['ym']
df.song = df.song.str.upper() + " - " + df.artist

In [400]:
df['ym'] = df.date.apply(date_to_ym)

In [401]:
df.sort_values(['weeks-on-board'], ascending=False)

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,ym
1719,2021-09-04,20,BLINDING LIGHTS - The Weeknd,The Weeknd,21.0,1,90,2021.09
1820,2021-08-28,21,BLINDING LIGHTS - The Weeknd,The Weeknd,18.0,1,89,2021.08
1917,2021-08-21,18,BLINDING LIGHTS - The Weeknd,The Weeknd,16.0,1,88,2021.08
2015,2021-08-14,16,BLINDING LIGHTS - The Weeknd,The Weeknd,17.0,1,87,2021.08
39948,2014-05-10,49,RADIOACTIVE - Imagine Dragons,Imagine Dragons,48.0,3,87,2014.05
...,...,...,...,...,...,...,...,...
21278,2017-12-09,79,AWFUL THINGS - Lil Peep,Lil Peep,0.0,79,1,2017.12
98775,2003-02-01,76,CAN'T STOP LOVING YOU - Phil Collins,Phil Collins,0.0,76,1,2003.02
59821,2010-07-17,22,POWER - Kanye West,Kanye West,0.0,22,1,2010.07
21246,2017-12-09,47,ECHAME LA CULPA - Luis Fonsi & Demi Lovato,Luis Fonsi & Demi Lovato,0.0,47,1,2017.12


In [402]:
df_final = df.sort_values(["ym", "weeks-on-board"], ascending = False).drop_duplicates(['ym', 'song']).reset_index(drop=True)

In [405]:
temp_df = df_final[df_final.ym == 2000.01]
init_df = temp_df.sort_values("weeks-on-board", ascending = False).head(10)[['ym','song', 'weeks-on-board']].reset_index(drop=True)
init_df

,ym,song,weeks-on-board
0,2000.01,WHERE MY GIRLS AT? - 702,40
1,2000.01,UNPRETTY - TLC,32
2,2000.01,SOMEDAY - Sugar Ray,31
3,2000.01,BLACK BALLOON - Goo Goo Dolls,30
4,2000.01,BACK THAT THANG UP - Juvenile Featuring Mannie...,30
5,2000.01,SMOOTH - Santana Featuring Rob Thomas,27
6,2000.01,THE ROCKAFELLER SKANK - Fatboy Slim,27
7,2000.01,AMAZED - Lonestar,26
8,2000.01,STEAL MY SUNSHINE - Len,25
9,2000.01,SEXUAL (LI DA DI) - Amber,25


In [406]:
init_df

,ym,song,weeks-on-board
0,2000.01,WHERE MY GIRLS AT? - 702,40
1,2000.01,UNPRETTY - TLC,32
2,2000.01,SOMEDAY - Sugar Ray,31
3,2000.01,BLACK BALLOON - Goo Goo Dolls,30
4,2000.01,BACK THAT THANG UP - Juvenile Featuring Mannie...,30
5,2000.01,SMOOTH - Santana Featuring Rob Thomas,27
6,2000.01,THE ROCKAFELLER SKANK - Fatboy Slim,27
7,2000.01,AMAZED - Lonestar,26
8,2000.01,STEAL MY SUNSHINE - Len,25
9,2000.01,SEXUAL (LI DA DI) - Amber,25


In [407]:
best_df = init_df[["song", "weeks-on-board"]]
best_song = list(best_df['song'])
best_val = list(best_df['weeks-on-board'])
best_val

[40, 32, 31, 30, 30, 27, 27, 26, 25, 25]

In [408]:
best_song

['WHERE MY GIRLS AT? - 702',
 'UNPRETTY - TLC',
 'SOMEDAY - Sugar Ray',
 'BLACK BALLOON - Goo Goo Dolls',
 "BACK THAT THANG UP - Juvenile Featuring Mannie Fresh & Lil' Wayne",
 'SMOOTH - Santana Featuring Rob Thomas',
 'THE ROCKAFELLER SKANK - Fatboy Slim',
 'AMAZED - Lonestar',
 'STEAL MY SUNSHINE - Len',
 'SEXUAL (LI DA DI) - Amber']

Keep a sorted list of best song, loop through all months.

In [414]:
rows = []
smooth_occur = False

for y in range(2000, 2022):
     for m in range(1, 13): 
          cur_ym = float(str(y)) + float(str(m/100))
          temp_df = df[df.ym == cur_ym]
          # print(temp_df)
          for index, row in temp_df.iterrows():
               song, val = row['song'], row['weeks-on-board']
               if song in best_song:
                    # update song's val
                    best_val[best_song.index(song)] = val
                    # must re-sort
                    best_song = [x for _, x in sorted(zip(best_val, best_song), reverse = True)]
                    best_val = [x for x, _ in sorted(zip(best_val, best_song), reverse = True)]
                    
                    

               elif val > best_val[-1]:
                    for i, bv in enumerate(best_val):
                         if val > bv:
                              
                                   
                              best_val.insert(i, val)
                              best_song.insert(i, song)

                              best_val = best_val[:-1]
                              best_song = best_song[:-1]
                                   
                              break
          # if y == 2006:
          #      print('month', m)
          #      if "Smooth" in best_song:
          #           print("Smooth in")
          #           print("best song",best_song)
          #           print("best val", best_val)
          #      else:
          #           print("Smooth not in")
          #           print("best song",best_song)
          #           print("best val", best_val)
                         
          new_row = []
          
          # print(len(best_song),"make sure its 10")
          for new_song, new_val in zip(best_song, best_val):
               new_row.append([cur_ym, new_song, new_val])
          rows += new_row
     #      break
     # break
nice_df = pd.DataFrame(rows, columns=["year", "name", "value"])
# row: year, song, weeks-on-board


In [410]:
nice_df.sort_values(['year','value'], ascending = False).groupby("year").head(1)

,year,name,value
2630,2021.12,BLINDING LIGHTS - The Weeknd,90
2620,2021.11,BLINDING LIGHTS - The Weeknd,90
2610,2021.10,BLINDING LIGHTS - The Weeknd,90
2600,2021.09,BLINDING LIGHTS - The Weeknd,90
2590,2021.08,RADIOACTIVE - Imagine Dragons,86
...,...,...,...
40,2000.05,WHERE MY GIRLS AT? - 702,41
30,2000.04,WHERE MY GIRLS AT? - 702,41
20,2000.03,WHERE MY GIRLS AT? - 702,41
10,2000.02,WHERE MY GIRLS AT? - 702,41


In [412]:
nice_df["lastValue"] = nice_df.value

In [413]:
nice_df.to_csv("nice_df_with_artist.csv", index=False)

In [274]:
nice_df.drop_duplicates("year").name.value_counts()

I'm Yours             144
Higher                 65
You And Me             38
Where My Girls At?     10
Blinding Lights         7
Name: name, dtype: int64

In [182]:
for i, j in zip(best_song, best_val):
    print(i,j)

Black Balloon 26
Kryptonite 27
Higher 17
Breathe 9
I Wanna Know 1
Everything You Want 1
Where My Girls At? 36
He Wasn't Man Enough 30
Try Again 30
There You Go 32
